In [15]:
from PIL import Image as PIL_Image
import cv2
import imutils
import matplotlib.pyplot as plt
import numpy as np
import easyocr
import pytesseract

In [18]:
orig_img_path = 'data/car1.jpg'
img = cv2.imread(orig_img_path)
img = cv2.resize(img, (620,480) )

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert to grey scale
gray = cv2.bilateralFilter(gray, 11, 17, 17)

edged = cv2.Canny(gray, 30, 200) #Perform Edge detection
print(type(edged))
cv2.imshow("windows", edged)
print(len(edged))
key = cv2.waitKey(0)
# if key == ord('q'):
#    print("exit")
#cv2.imwrite("output.jpg", img)
cv2.destroyWindow("windows")


<class 'numpy.ndarray'>
480


In [4]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]
print(type(cnts))
print(len(cnts))
screenCnt = None

<class 'list'>
10


In [13]:
# loop over our contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)
    print("type(approx):",type(approx))
    # if our approximated contour has four points, then
    # we can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        print("type(screenCnt)",type(screenCnt))
        print(len(screenCnt))
        break

type(approx): <class 'numpy.ndarray'>
type(approx): <class 'numpy.ndarray'>
type(screenCnt) <class 'numpy.ndarray'>
4


In [6]:
cv2.imwrite("output.jpg", screenCnt)

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:699: error: (-215:Assertion failed) image.channels() == 1 || image.channels() == 3 || image.channels() == 4 in function 'cv::imwrite_'


In [ ]:
cv2.imshow("windows1", screenCnt)
key = cv2.waitKey(0)
# if key == ord('q'):
#    print("exit")

cv2.destroyWindow("windows1")

In [ ]:
type(cnts)

In [4]:
import cv2
import imutils
import numpy as np
import pytesseract
from PIL import Image

#img = cv2.imread('data/plate1.jpg',cv2.IMREAD_COLOR)
img = cv2.imread('data/car1.jpg',cv2.IMREAD_COLOR)
img = cv2.resize(img, (600,400))

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert to grey scale
gray = cv2.bilateralFilter(gray, 11, 17, 17) #Blur to reduce noise

edged = cv2.Canny(gray, 30, 200) #Perform Edge detection
cv2.imshow('edged',edged)

# find contours in the edged image, keep only the largest
# ones, and initialize our screen contour
cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]

# max_weight=0
# for item in cnts:
#     x,y,weight,height = cv2.boundingRect(item)
#     print(x,y,weight,height)    
#     if (weight / height > 1.5) and (weight>max_weight):
#         max_weight=weight
#         crop_img = gray[y-20:y + height + 15, x - 5:x + weight+15]


screenCnt = None
# loop over our contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.020 * peri, True)
    
 # if our approximated contour has four points, then
 # we can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break
        
if screenCnt is None:
    detected = 0
    print("No contour detected")

else:
    detected = 1

if detected == 1:
    cv2.drawContours(img, [screenCnt], -1, (0, 255, 0), 3)


# Masking the part other than the number plate
mask = np.zeros(gray.shape,np.uint8)
new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
new_image = cv2.bitwise_and(img,img,mask=mask)


# Now crop
(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx+1, topy:bottomy+1]

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\\tesseract.exe'
#Read the number plate
text = pytesseract.image_to_string(Cropped, config='--psm 11')
print("Detected Number is:",text)

# cv2.imshow('edged',edged)
# cv2.imshow('image',img)
# cv2.imshow('Cropped',Cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()



Detected Number is: 


In [2]:
import cv2
import imutils
import numpy as np
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
img = cv2.imread('data/plate1',cv2.IMREAD_COLOR)
#img = cv2.resize(img, (600,400) )
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 13, 15, 15)
edged = cv2.Canny(gray, 30,150)
cv2.imshow('edged',edged)
cv2.waitKey(0)
contours=cv2.findContours(edged.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours,key=cv2.contourArea, reverse = True)[:20]
screenCnt = None
for c in contours:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)
    if len(approx) == 4:
        screenCnt = approx
        break
print(screenCnt)
mask = np.zeros(gray.shape,np.uint8)
new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
new_image = cv2.bitwise_and(img,img,mask=mask)
cv2.imshow('new_image',new_image)
cv2.waitKey(0)
(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx+1, topy:bottomy+1]
text = pytesseract.image_to_string(Cropped, config='--psm 11')
print("programming_fever's License Plate Recognition\n")
print("Detected license plate Number is:",text)
img = cv2.resize(img,(500,300))
Cropped = cv2.resize(Cropped,(400,200))
cv2.imshow('car',img)
cv2.imshow('Cropped',Cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
